In [55]:
MODEL = "llama-3.1-8b-inst"

gpu_point = "http://127.0.0.1:8000/v1/chat/completions"


import requests
import os
from dotenv import load_dotenv

load_dotenv("../.env")
from mem0 import MemoryClient
from multion.client import MultiOn

memo_client_key = os.environ["MEMO_KEY"]
MULTION_API_KEY = os.environ["MULTION_API_KEY"]
m_client = MemoryClient(api_key=memo_client_key)
multion = MultiOn(api_key=MULTION_API_KEY)


def config_llm(
    temperature: float = 0.6,
    max_tokens: int = 512,
    top_p: float = 0.8,
    presence_penalty=1.25,
    frequency_penalty=1.5,
) -> dict:
    return {
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "presence_penalty": presence_penalty,
        "frequency_penalty": frequency_penalty,
    }


def make_request_headers() -> dict:
    return {
        "accept": "application/json",
        "Content-Type": "application/json",
    }


def post_request(source: str, headers: dict, data: dict) -> dict:
    return requests.post(source, headers=headers, data=json.dumps(data)).json()


def browsing(command: str, url: str = "") -> str:
    browse_result = multion.browse(cmd=command, url=url)
    print(browse_result)
    return browse_result


def make_request_data(message_list) -> dict:
    return {
        "model": MODEL,
        "messages": message_list,
        "max_tokens": config_llm()["max_tokens"],
        "temperature": config_llm()["temperature"],
        "presence_penalty": config_llm()["presence_penalty"],
    }


import json


def message(by, to):
    return dict(zip(["role", "content"], [by, to]))


def message_system(to):
    return message("system", to)


def message_user(to):
    return message("user", to)


def message_assistant(to):
    return message("assistant", to)


def message_ipython(to):
    return message("ipython", to)


def get_response_completion(message_list) -> str:
    response: dict = post_request(
        source=gpu_point,
        headers=make_request_headers(),
        data=make_request_data(message_list=message_list),
    )
    buff = message_list
    content = response["choices"][0]["message"]["content"]
    if "<|python_tag|>" in content:
        try:
            call = exec(content.replace("<|python_tag|>", ""))
            buff.append(message_ipython(f"{call}"))
            return post_request(
                source=gpu_point,
                headers=make_request_headers(),
                data=make_request_data(message_list=buff),
            )["choices"][0]["message"]["content"]
        except Exception as e:
            print(e)
            return content
    else:
        return content


def browsing(command: str, url: str = "") -> str:
    browse_result = multion.browse(cmd=command)
    print(browse_result)
    return browse_result.message

In [56]:
import json
messages = []
# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append(message_system("""
Environment: ipython
Tools: brave_search, wolfram_alpha
"""))
messages.append(message_user("execute `2+2` using python interpreter"))
r = get_response_completion(messages)


4


In [52]:
r

'<|python_tag|>print(2+2)'

In [49]:
exec("print(2+2)")

4
